# COGS 108 - Data Checkpoint

# Names

- Shushruth Kallutla (A15542022)   
- Nicholas Peterzell (A15278235)  
- Amanda Mark (A15953259)  
- Taylor Drennen (A15460025)  
- Carter Nielsen (A15766670) 

<a id='research_question'></a>
# Research Question

Does Covid-19 inoculation rate in California correlate with the average income in a county? 

# Dataset(s)


- Dataset Name: **Personal Income and and Employment by County and Metropolitan Area**
- Link to the dataset: <https://apps.bea.gov/iTable/iTable.cfm?reqid=70&step=1&acrdn=6>
- Number of observations: 9414
-This dataset contains the names of all the counties in each state, total personal income (in thousands of dollars) ,population, and per capita personal income( in dollars) of each county  in 2019. e will use this dataset to extract the per capita income of all the counties in California. 
<br />
<br />
- Dataset Name: **Vaccine Administration by County of Residence**
- Link to the dataset: <https://covid19.ca.gov/vaccines/#California-vaccines-dashboard>
- Number of observations: 58
- This dataset gives a report of the amount of Covid 19 vaccines administered by the county. The bias with this dataset is that it counts people outside of California that got vaccinated in stated count; therefore, this data may not be a complete representation of state-wide vaccination. We will use this dataset to see the number of vaccines distributed in each county of California.
<br />
<br />
From the first dataset, we will extract per capita income of all the counties in California, and from the second dataset, we will get the amount of vaccines administered by each county. Those two datasets will be used to determine whether there is correlation between inoculation rate of Covid 19 vaccine and the income level of each county. 


# Setup

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

Income data from https://apps.bea.gov/iTable/iTable.cfm?reqid=70&step=1&acrdn=6

In [3]:
income_raw = pd.read_csv("Datasets/IncomeByCounty.csv")

In [4]:
income_raw.info()
income_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9414 entries, 0 to 9413
Data columns (total 5 columns):
GeoFips        9414 non-null int64
GeoName        9414 non-null object
LineCode       9414 non-null int64
Description    9414 non-null object
2019           9414 non-null object
dtypes: int64(2), object(3)
memory usage: 367.8+ KB


,GeoFips,GeoName,LineCode,Description,2019
0,1001,"Autauga, AL",1,Personal income (thousands of dollars),2453617
1,1001,"Autauga, AL",2,Population (persons) 1/,55869
2,1001,"Autauga, AL",3,Per capita personal income (dollars) 2/,43917
3,1003,"Baldwin, AL",1,Personal income (thousands of dollars),10600260
4,1003,"Baldwin, AL",2,Population (persons) 1/,223234


Vaccine Data from https://covid19.ca.gov/vaccines/#California-vaccines-dashboard  
Data from 2/11/2021 11:59pm

In [5]:
vaccine_count = pd.read_csv("Datasets/DosesAdminsteredbyCounty(California).csv")

In [6]:
vaccine_count.info()
vaccine_count.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 2 columns):
County                58 non-null object
Doses Administered    58 non-null object
dtypes: object(2)
memory usage: 1008.0+ bytes


,County,Doses Administered
0,Los Angeles,"1,314,016"
1,San Diego,"532,215"
2,Orange,"438,774"
3,Santa Clara,"271,559"
4,Riverside,"257,303"


# Data Cleaning

### Income Data Cleaning
Row data different depending on linecode, split different linecodes into income and population rows Need county population and county income as columns and can remove other columns

In [7]:
#Seperate data by linecode value
group = income_raw.groupby("LineCode")

counties = group.get_group(1)["GeoName"].reset_index()
del counties["index"]


income_data = group.get_group(1)["2019"].reset_index()
del income_data["index"]

population_data = group.get_group(2)["2019"].reset_index()
del population_data["index"]

In [8]:
county_data = pd.DataFrame(counties)
county_data = county_data.rename(columns={"GeoName": "County"})

county_data["Total Income"] = income_data
county_data["Population"] = population_data

county_data.head()

,County,Total Income,Population
0,"Autauga, AL",2453617,55869
1,"Baldwin, AL",10600260,223234
2,"Barbour, AL",882834,24686
3,"Bibb, AL",710451,22394
4,"Blount, AL",2105550,57826


In [9]:
#Remove NA from Dataset for accurate int datatype
county_data = county_data.replace("(NA)", "1")
#Convert object data to int
county_data["Total Income"] = pd.to_numeric(county_data["Total Income"])
county_data["Population"] = pd.to_numeric(county_data["Population"])

county_data.dtypes.head()

County          object
Total Income     int64
Population       int64
dtype: object

In [10]:
#Calculate Income per Capita
percapitaincome = county_data.apply(lambda row: row["Total Income"] / row["Population"], axis = 1).reset_index()
del percapitaincome["index"]
county_data["Income per Capita"] = percapitaincome
county_data.to_csv("CleanedIncomePerCapita.csv")
county_data.head()

,County,Total Income,Population,Income per Capita
0,"Autauga, AL",2453617,55869,43.917324
1,"Baldwin, AL",10600260,223234,47.484971
2,"Barbour, AL",882834,24686,35.762537
3,"Bibb, AL",710451,22394,31.725060
4,"Blount, AL",2105550,57826,36.411822


In [11]:
#Total Income and Income per Capita columns are in the thousands,
#so it would be easier to convert them to an accurate count
county_data["Total Income"] = 1000 *county_data["Total Income"]
county_data["Income per Capita"] = 1000 * county_data["Income per Capita"]
county_data.head()

,County,Total Income,Population,Income per Capita
0,"Autauga, AL",2453617000,55869,43917.324455
1,"Baldwin, AL",10600260000,223234,47484.970927
2,"Barbour, AL",882834000,24686,35762.537471
3,"Bibb, AL",710451000,22394,31725.060284
4,"Blount, AL",2105550000,57826,36411.821672


In [12]:
#Seperate County and State
count_state_temp = county_data["County"].str.split(", ", n = 1, expand = True) 
county_data.drop(columns =["County"], inplace = True) 

county_data["County"] = count_state_temp[0]
county_data["State"] = count_state_temp[1]

In [13]:
county_data.head()

,Total Income,Population,Income per Capita,County,State
0,2453617000,55869,43917.324455,Autauga,AL
1,10600260000,223234,47484.970927,Baldwin,AL
2,882834000,24686,35762.537471,Barbour,AL
3,710451000,22394,31725.060284,Bibb,AL
4,2105550000,57826,36411.821672,Blount,AL


In [13]:
#filter only california counties
cal_income = county_data[(county_data.State == "CA")]
cal_income.head()

,Total Income,Population,Income per Capita,County,State
210,135663560000,1671329,81171.068054,Alameda,CA
211,81463000,1129,72155.004429,Alpine,CA
212,1825410000,39752,45919.953713,Amador,CA
213,10490209000,219186,47859.849625,Butte,CA
214,2347168000,45905,51130.987910,Calaveras,CA


In [14]:
#Removing the state column now that all samples are from CA
cal_income = cal_income.drop(["State"], axis=1)
cal_income.head()

,Total Income,Population,Income per Capita,County
210,135663560000,1671329,81171.068054,Alameda
211,81463000,1129,72155.004429,Alpine
212,1825410000,39752,45919.953713,Amador
213,10490209000,219186,47859.849625,Butte
214,2347168000,45905,51130.987910,Calaveras


In [21]:
#Moving the County column to the first position
cal_income = cal_income[['County', 'Total Income', 'Population', 'Income per Capita']]
cal_income.head()

,County,Total Income,Population,Income per Capita
210,Alameda,135663560000,1671329,81171.068054
211,Alpine,81463000,1129,72155.004429
212,Amador,1825410000,39752,45919.953713
213,Butte,10490209000,219186,47859.849625
214,Calaveras,2347168000,45905,51130.987910


### Merge Datasets

In [22]:
merged = cal_income.merge(vaccine_count, "inner", "County")
merged.info()
merged.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58 entries, 0 to 57
Data columns (total 5 columns):
County                58 non-null object
Total Income          58 non-null int64
Population            58 non-null int64
Income per Capita     58 non-null float64
Doses Administered    58 non-null object
dtypes: float64(1), int64(2), object(2)
memory usage: 2.7+ KB


,County,Total Income,Population,Income per Capita,Doses Administered
0,Alameda,135663560000,1671329,81171.068054,"216,440"
1,Alpine,81463000,1129,72155.004429,362
2,Amador,1825410000,39752,45919.953713,"4,777"
3,Butte,10490209000,219186,47859.849625,"39,012"
4,Calaveras,2347168000,45905,51130.987910,"5,568"


# Project Proposal (updated)

## Project Timeline

Meeting Date | Completed Before Meeting | Discuss at Meeting
--- | --- | ---
1/22 | Read & Think about COGS 108 expectations; brainstorm topics/questions | Determine best form of communication; Discuss and decide on final project topic; discuss hypothesis; begin background research  
1/25 | Do background research on topic | Discuss ideal dataset(s) and ethics; draft project proposal  
1/29 | Edit, finalize, and submit proposal; Search for datasets | Discuss Wrangling and possible analytical approaches; Assign group members to lead each specific part  
2/12 | Import & Wrangle Data  | Review/Edit wrangling/EDA; Discuss Analysis Plan  
2/26 | Finalize wrangling/EDA | Discuss/edit Analysis; Complete project check-in  
3/12 | Complete analysis; Draft results/conclusion/discussion | Discuss/edit full project
3/17 | N/A | Turn in Final Project & Group Project Surveys